In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ── Hamilton Star / PyLabRobot: dispense 20 µL dye into qPCR plate ──
#
# Hardware assumptions
#  • Hamilton STARlet (8-channel) with CO-RE head
#  • Autoload carousel present
#  • “core gripper” detached
#
# Deck layout
#  ────────── rails 1 … 20 ──────────
#   1  ─ TIP_CAR_480_A00  – HTF 300 µL tips in pos 0, 50 µL filter tips in pos 1
#  13  ─ MFX_CAR_L5_base  – slot 0: MFX_DWP_module_188042
#                           ↳ on that module: opentrons_24_tuberack_VWR_2point0ml_snapcap_short
#                                (tube D1 contains ~2 mL Cy5 dye)
#  19  ─ MFX_CAR_L5_base  – slot 0: MFX_DWP_module_188042
#                           ↳ on that module: VWR_96_wellplate_100_Vb   (target plate)
#
# Execution steps
#  1. setup()   (skip autoload calibration for speed while prototyping)
#  2. pick up a single 50 µL filter tip on channel 0
#  3. pre-wet:  aspiration + dispense 30 µL from dye tube D1, 3×
#  4. aspirate 220 µL dye (extra headroom)  → dispense 20 µL into each well
#     ▸ slower speeds and a short post-dispense delay for accuracy
#  5. blow out residual dye into tube D1
#  6. discard tip to waste (position C4 of the tip rack for simplicity)
#
#  7. print lh.summary() so you can confirm volumes, locations, etc.
#  8. optional: lh.visualize()   (opens an interactive deck map in notebook)

################################################################################


import asyncio
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources.hamilton import (
    STARLetDeck, MFX_CAR_L5_base, TIP_CAR_480_A00
)
from pylabrobot.resources.hamilton.mfx_modules import MFX_DWP_module_188042
from pylabrobot.resources import HTF, TIP_50ul_w_filter





In [3]:
#labware definitions for this experiment
from pylabrobot.resources.height_volume_functions import (
  calculate_liquid_height_in_container_2segments_square_vbottom,
  calculate_liquid_volume_container_2segments_square_vbottom,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)
from pylabrobot.resources.height_volume_functions import (
  calculate_liquid_height_in_container_2segments_square_vbottom,
  calculate_liquid_volume_container_2segments_square_vbottom,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)
def VWR_96_wellplate_100_Vb(name: str, with_lid: bool = False) -> Plate:
  """
This plate is a VWR PCR plate 96 well low-profile, half-skirted, ABI-FAST type plate.
VWR cat no. 89218-296
It is half-skirted so it must reside in another plate like a Cor_96_wellplate_360ul_Fb
  """
  
  return Plate(
    name=name,
    size_x=127.76,
    size_y=85.48,
    size_z=20.0,
    # lid=lid,
    model=VWR_96_wellplate_100_Vb.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=10.25,  # keeping costar measurement
      dy=10.5,  # 7.77 keeping costar measurement
      dz=8.5, # how high is well above base
      item_dx=9.0,
      item_dy=9.0,
      size_x=5.4,  # measured
      size_y=5.4,  # measured
      size_z=16.3, # measured well depth, costar + VWR plate height
      material_z_thickness=0.5,
      bottom_type=WellBottomType.V,
      cross_section_type=CrossSectionType.CIRCLE,
      max_volume=100,
    ),
  )

from typing import Optional

from pylabrobot.resources.height_volume_functions import (
  compute_height_from_volume_rectangle,
  compute_volume_from_height_rectangle,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)

def opentrons_24_tuberack_VWR_2point0ml_snapcap_short(name: str, lid: Optional[Lid] = None) -> Plate:
  """
  OpenTrons 24 well rack with the shorter stand
  3D print available here: https://www.thingiverse.com/thing:3405002
  Spec sheet (json):
  https://raw.githubusercontent.com/Opentrons/opentrons/edge/shared-data/labware/definitions/2/opentrons_24_tuberack_nest_1.5ml_screwcap/1.json
  VWR 2.0mL graduated tubes Cat. no. 20170-170
  """
  INNER_WELL_WIDTH = 9.0  # measured  
  INNER_WELL_LENGTH = 9.0  # measured

  well_kwargs = {
    "size_x": INNER_WELL_WIDTH,  # measured
    "size_y": INNER_WELL_LENGTH,  # measured
    "size_z": 39.5,  # measured
    "bottom_type": WellBottomType.V,
    "cross_section_type": CrossSectionType.CIRCLE,
    "compute_height_from_volume": lambda liquid_volume: compute_height_from_volume_rectangle(
      liquid_volume,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    "compute_volume_from_height": lambda liquid_height: compute_volume_from_height_rectangle(
      liquid_height,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    # "material_z_thickness": 1,
    "material_z_thickness": 0.80, # measured
  }

  return Plate(
    name=name,
    size_x=127.75,  # from spec
    size_y=85.50,  # from spec
    size_z=48.5,  # measured (this is the shorter platform)
    lid=lid,
    model=opentrons_24_tuberack_VWR_2point0ml_snapcap_short.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=6,
      num_items_y=4,
      dx=12.5,  # measured
      dy=16.5,  # measured
      dz=17,  # measured
      item_dx=19.89, # from spec
      item_dy=19.28, # from spec
      **well_kwargs,
    ),
  )


In [4]:
# lh.summary()

In [5]:
# # ── build LH + empty deck ─────────────────────────────────────────────────────
# backend = STARBackend()
# lh      = LiquidHandler(backend=backend, deck=STARLetDeck())
# await lh.setup(skip_autoload=True)          # faster while developing


# # ── TIP CARRIER on rail 1 ──
# tip_car = TIP_CAR_480_A00("tip_car")
# tip_car[1] = TIP_50ul_w_filter(name="tips_50f")  # will use pos A-row first
# lh.deck.assign_child_resource(tip_car, rails=1)

# # ── SOURCE carrier @ rail 13 ──
# dwp_mod_src   = MFX_DWP_module_188042("dwp_mod_src")
# flex_car_src  = MFX_CAR_L5_base("flex_car_src", modules={0: dwp_mod_src})
# lh.deck.assign_child_resource(flex_car_src, rails=13)

# tube_rack = opentrons_24_tuberack_VWR_2point0ml_snapcap_short("dye_rack")
# dwp_mod_src.assign_child_resource(tube_rack, location=0)

# # Cy5 dye tube in rack position D1 (4th row, 1st column)
# dye_tube = tube_rack["D1"]

# # ── TARGET carrier @ rail 19 ──
# dwp_mod_tgt   = MFX_DWP_module_188042("dwp_mod_tgt")
# flex_car_tgt  = MFX_CAR_L5_base("flex_car_tgt", modules={0: dwp_mod_tgt})
# lh.deck.assign_child_resource(flex_car_tgt, rails=19)

# plate = VWR_96_wellplate_100_Vb("qpcr_plate")
# dwp_mod_tgt.assign_child_resource(plate, location=0)
# plate = lh.deck.get_resource("qpcr_plate")   # returns the *located* object

# print("plate location →", plate.location)          # should be Coordinate(...)
# print("module parent  →", plate.parent.name)       # 'dwp_mod_tgt'


# ################################################################################
# # Liquid class parameters tuned for accuracy over speed
# ################################################################################

# # ── liquid-class settings focused on precision ───────────────────────────────
# precise = dict(
#     # liquid_class="Water_Disp_Acc",   # Hamilton accurate aqueous LC
#     aspirate_flow_rate=10,           # µL s⁻¹
#     dispense_flow_rate=10,
#     end_delay_seconds=0.5,
#     blow_out=[0]
# )

# # Hamilton-specific LLD parameters you’ll merge in when you aspirate
# lld_surface = dict(
#     lld_mode=[STARBackend.LLDMode.GAMMA],        # pressure LLD = best for disposable tips
#     # lld_search_height=[40],       # start 30 mm above deck, move down until hit
#     # immersion_depth=[1],          # go 1 mm BELOW detected surface
#     # immersion_depth_direction=[0]   ## 0 = go below surface
#     # gamma_lld_sensitivity=[4]
# )

# # ################################################################################
# # Main protocol
# ################################################################################

# async def run():
#     # 1  pick up one filtered 50 µL tip
#     await lh.pick_up_tips(lh.deck.get_resource("tips_50f")["E1"])

#     # 2  3× pre-wet (30 µL up & down)
#     for _ in range(3):
#         await lh.aspirate(dye_tube, vols=[30], use_channels=[0],**precise, **lld_surface)
#         await lh.dispense(dye_tube, vols=[30], use_channels=[0], **precise, liquid_height=[30] )

#     # 3  distribute 20 µL to every well A1–H12
#     for well in plate["A1:H12"]:
#         await lh.aspirate(dye_tube, vols=[20], use_channels=[0], **precise, **lld_surface)
#         # await lh.aspirate(dye_tube, vols=[20], use_channels=[0], **precise)   # 5 µL headroom
#         await lh.dispense([well],    vols=[20], use_channels=[0], **precise)

#     # # 4) blow out residual dye into source tube
#     # await lh.dispense(dye_tube, vols=[0], use_channels=[0], blow_out=[1])

#     # 5) discard tip
#     await lh.discard_tips()




# # ── run ───────────────────────────────────────────────────────────────────────
# await run()


In [11]:
# """
# Augmented PyLabRobot protocol: pair‑wise 20 µL dispenses, static z (no polish / no rise).

# Revision (per user 2025‑07‑16):
# • Remove the bottom‑tracking / polish step (no dynamic move during dispense).
# • Dispense 20 µL into each destination well at a fixed **liquid_height = 2 mm above well bottom**.
# • Still operate in 2‑well (adjacent) aliquot pairs per aspiration cycle: 50 µL prime for the first pair, then cycles of 40 µL top‑ups riding on ~10 µL residual to give 20 µL + 20 µL each time.
# • Repeat across the full 96‑well plate (A1→H12 in row‑major order; wells are paired sequentially so [A1,A2], [A3,A4], …, [H11,H12]).

# Comment on `vols=[0]` blow‑out step at end (unchanged):
# In PyLabRobot (Hamilton backend), a dispense with `vols=[0]` + `blow_out=[1]` performs a blow‑out of residual volume without metering an additional programmed volume. This clears whatever is left in the tip (~10 µL) plus LC‑defined blow‑out air. Change to a non‑zero volume only if you truly want to meter that amount back; otherwise you may push air and corrupt volume accounting.
# """

# ###############################################################################
# # ‑‑ imports ‑‑
# ###############################################################################

# from pylabrobot.liquid_handling import LiquidHandler, STARBackend


# ##############################################################################
# ##‑‑ build LH + empty deck ‑‑
# ##############################################################################
# backend = STARBackend()
# lh      = LiquidHandler(backend=backend, deck=STARLetDeck())
# await lh.setup(skip_autoload=True)  # faster during development

# # ‑‑ TIP CARRIER on rail 1 ‑‑
# tip_car = TIP_CAR_480_A00("tip_car")
# tip_car[1] = TIP_50ul_w_filter(name="tips_50f")  # uses A‑row first
# lh.deck.assign_child_resource(tip_car, rails=1)

# # ‑‑ SOURCE carrier @ rail 13 ‑‑
# dwp_mod_src   = MFX_DWP_module_188042("dwp_mod_src")
# flex_car_src  = MFX_CAR_L5_base("flex_car_src", modules={0: dwp_mod_src})
# lh.deck.assign_child_resource(flex_car_src, rails=13)

# tube_rack = opentrons_24_tuberack_VWR_2point0ml_snapcap_short("dye_rack")
# dwp_mod_src.assign_child_resource(tube_rack, location=0)

# dye_tube = tube_rack["D1"]  # Cy5 dye source

# # ‑‑ TARGET carrier @ rail 19 ‑‑
# dwp_mod_tgt   = MFX_DWP_module_188042("dwp_mod_tgt")
# flex_car_tgt  = MFX_CAR_L5_base("flex_car_tgt", modules={0: dwp_mod_tgt})
# lh.deck.assign_child_resource(flex_car_tgt, rails=19)

# plate = VWR_96_wellplate_100_Vb("qpcr_plate")
# dwp_mod_tgt.assign_child_resource(plate, location=0)
# plate = lh.deck.get_resource("qpcr_plate")  # located object

###############################################################################
# ‑‑ liquid‑class parameters tuned for accuracy over speed ‑‑
###############################################################################

precise = dict(
    # liquid_class="Water_Disp_Acc",   # Hamilton accurate aqueous LC
    aspirate_flow_rate=10,           # µL s⁻¹
    dispense_flow_rate=10,
    end_delay_seconds=0.5,
    blow_out=[0]                     # default no blow‑out unless overridden
)

lld_surface = dict(
    lld_mode=[STARBackend.LLDMode.GAMMA],
    immersion_depth=[2]
    
)

###############################################################################
# ‑‑ helper constants ‑‑
###############################################################################
CHANNEL = 2  # single‑channel work for clarity; adapt for multichannel if desired

###############################################################################
# ‑‑ main protocol ‑‑
###############################################################################

async def run():
    # 1) pick up filtered 50 µL tip
    await lh.pick_up_tips(lh.deck.get_resource("tips_50f")["C8"], use_channels=[CHANNEL])

    # 2) 3× pre‑wet (30 µL up & down)
    for _ in range(3):
        await lh.aspirate(dye_tube, vols=[30], use_channels=[CHANNEL], **precise, **lld_surface)
        await lh.dispense(dye_tube, vols=[30], use_channels=[CHANNEL], **precise, liquid_height=[30])

    # 3) prime tip with 50 µL before first dispense pair
    await lh.aspirate(dye_tube, vols=[50], use_channels=[CHANNEL], **precise, **lld_surface)

    # 4) build full 96‑well list (row‑major), then iterate in adjacent pairs
    dest_wells = list(plate["A1:H12"])  # PLR slice expands to 96 wells A1..H12 row‑major

    for i in range(0, len(dest_wells), 2):
        w1 = dest_wells[i]
        w2 = dest_wells[i+1]

        # —— dispense 20 µL into first well @ liquid_height=2 mm (static, no move)
        await lh.dispense([w1], vols=[20], use_channels=[CHANNEL], **precise, liquid_height=[2])

        # —— dispense 20 µL into second well @ liquid_height=2 mm
        await lh.dispense([w2], vols=[20], use_channels=[CHANNEL], **precise, liquid_height=[2])

        # —— aspirate 40 µL fresh dye (assumes ~10 µL residual remains from prior step)
        if i + 2 < len(dest_wells):  # skip after last pair
            try:
                await lh.aspirate(dye_tube, vols=[40], use_channels=[CHANNEL], **precise, **lld_surface)
            except: # when volume too low
                await lh.aspirate(dye_tube, vols=[40], use_channels=[CHANNEL], **precise, liquid_height=[1]) 

    # Optional: recover residual 10 µL to source & blow out.
    # Comment out if you prefer to discard the tip with dye still in it.
    # await lh.dispense(dye_tube, vols=[0], use_channels=[CHANNEL], blow_out=[1])

    # 5) discard tip
    await lh.discard_tips()

###############################################################################
# ‑‑ execute ‑‑
###############################################################################
await run()


In [7]:

# # await lh.dispense(dye_tube, vols=[0], use_channels=[1], blow_out=[1])
# await lh.discard_tips()
# # # # await lh.prepare_for_manual_channel_operation(0)
# # await lh.stop()
# # # await lh.pick_up_tips(lh.deck.get_resource("tips_50f")["B7"], use_channels=CHANNEL)
# # tiprack = lh.deck.get_resource("tips_50f")
# # # await lh.pick_up_tips(tiprack["A1:C1"])
# # # tip_car[1] = TIP_50ul_w_filter(name="tips_50f") 
# # await lh.drop_tips(tiprack["B7"],use_channels=[CHANNEL])